In [ ]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px

# traigo mis funciones
import sys
sys.path.append('../') #pongo los 2 puntos para que salga de la carpeta
import src.funciones as f
import src.soporte_prep as spp
import src.soporte_encoding as se
# Métodos estadísticos
# -----------------------------------------------------------------------
from scipy.stats import zscore # para calcular el z-score
from sklearn.neighbors import LocalOutlierFactor # para detectar outliers usando el método LOF
from sklearn.ensemble import IsolationForest # para detectar outliers usando el metodo IF
from sklearn.neighbors import NearestNeighbors # para calcular la epsilon

# Para generar combinaciones de listas
# -----------------------------------------------------------------------
from itertools import product
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, TargetEncoder # para poder aplicar los métodos de OneHot, Ordinal,  Label y Target Encoder 

# Gestionar warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from category_encoders import TargetEncoder

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_pickle('../datos/df_estandarizado.pkl')

In [3]:
df.head()

,Age_robust,Attrition,BusinessTravel,Department,DistanceFromHome_robust,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome_robust,NumCompaniesWorked_robust,PercentSalaryHike_robust,StockOptionLevel,TotalWorkingYears_robust,TrainingTimesLastYear,YearsAtCompany_robust,YearsSinceLastPromotion_robust,YearsWithCurrManager_robust,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1.153846,0,Travel_Rarely,Sales,-0.083333,2,Life Sciences,Female,1,Healthcare Representative,Married,1.495430,-0.333333,-0.500000,0,-1.000000,6,-0.666667,-0.333333,-0.6,3.0,4.0,2.0,3,3
1,-0.384615,1,Travel_Frequently,Research & Development,0.250000,1,Life Sciences,Female,1,Research Scientist,Single,-0.136563,-0.666667,1.500000,1,-0.444444,3,0.000000,0.000000,0.2,3.0,2.0,4.0,2,4
2,-0.307692,0,Travel_Frequently,Research & Development,0.833333,4,Other,Male,4,Sales Executive,Married,2.631079,-0.333333,0.166667,3,-0.555556,2,0.000000,-0.333333,0.0,2.0,2.0,1.0,3,3
3,0.153846,0,Non-Travel,Research & Development,-0.416667,5,Life Sciences,Male,3,Human Resources,Married,0.618830,0.333333,-0.500000,3,0.333333,5,0.500000,2.000000,0.4,4.0,4.0,3.0,2,3
4,-0.307692,0,Travel_Rarely,Research & Development,0.250000,1,Medical,Male,1,Sales Executive,Single,-0.474223,0.666667,-0.333333,2,-0.111111,2,0.166667,-0.333333,0.2,4.0,1.0,3.0,3,3


In [7]:
lista_col_cat = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [9]:
lista_col_num = df.select_dtypes(include=[int, float]).columns.tolist()

In [10]:
len(lista_col_cat)

15

In [11]:
len(lista_col_num)

10

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4389 entries, 0 to 4409
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Age_robust                      4389 non-null   float64 
 1   Attrition                       4389 non-null   int64   
 2   BusinessTravel                  4389 non-null   object  
 3   Department                      4389 non-null   object  
 4   DistanceFromHome_robust         4389 non-null   float64 
 5   Education                       4389 non-null   category
 6   EducationField                  4389 non-null   object  
 7   Gender                          4389 non-null   object  
 8   JobLevel                        4389 non-null   category
 9   JobRole                         4389 non-null   object  
 10  MaritalStatus                   4389 non-null   object  
 11  MonthlyIncome_robust            4389 non-null   float64 
 12  NumCompaniesWorked_robust

In [22]:
df['EnvironmentSatisfaction'] = df['EnvironmentSatisfaction'].astype(str)
df['JobSatisfaction'] = df['JobSatisfaction'].astype(str)
df['WorkLifeBalance'] = df['WorkLifeBalance'].astype(str)


In [23]:
lista_ord, lista_des = f.detectar_orden_cat(df, lista_cat=df.select_dtypes(include=['object', 'category']), var_respuesta='Attrition')

Estamos evaluando la variable BUSINESSTRAVEL


Attrition,0,1
BusinessTravel,,
Non-Travel,411,36
Travel_Frequently,624,207
Travel_Rarely,2646,465


Sí tiene orden la variable BusinessTravel
Estamos evaluando la variable DEPARTMENT


Attrition,0,1
Department,,
Human Resources,132,57
Research & Development,2418,450
Sales,1131,201


Sí tiene orden la variable Department
Estamos evaluando la variable EDUCATION


...

Sí tiene orden la variable WorkLifeBalance
Estamos evaluando la variable JOBINVOLVEMENT


Attrition,0,1
JobInvolvement,,
1,192,54
2,942,180
3,2196,396
4,351,78


Sí tiene orden la variable JobInvolvement
Estamos evaluando la variable PERFORMANCERATING


Attrition,0,1
PerformanceRating,,
3,3129,585
4,552,123


La variable PerformanceRating no tiene orden


In [31]:
lista_ord

['BusinessTravel',
 'Department',
 'EducationField',
 'JobRole',
 'MaritalStatus',
 'TrainingTimesLastYear',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'WorkLifeBalance',
 'JobInvolvement']

In [33]:
lista_des

['Education', 'Gender', 'JobLevel', 'StockOptionLevel', 'PerformanceRating']

In [32]:
diccionario_encoding = { "target": lista_ord, 
                        "onehot": lista_des} 
# En caso de que queramos aplicar el método solo a una columna
encoding = se.Encoding(df, variable_respuesta="Attrition", diccionario_encoding=diccionario_encoding)
df_resultado = encoding.one_hot_encoding()
df_resultado = encoding.target_encoding()
df.head()

,Age_robust,Attrition,BusinessTravel,Department,DistanceFromHome_robust,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome_robust,NumCompaniesWorked_robust,PercentSalaryHike_robust,StockOptionLevel,TotalWorkingYears_robust,TrainingTimesLastYear,YearsAtCompany_robust,YearsSinceLastPromotion_robust,YearsWithCurrManager_robust,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1.153846,0,Travel_Rarely,Sales,-0.083333,2,Life Sciences,Female,1,Healthcare Representative,Married,1.495430,-0.333333,-0.500000,0,-1.000000,6,-0.666667,-0.333333,-0.6,3.0,4.0,2.0,3,3
1,-0.384615,1,Travel_Frequently,Research & Development,0.250000,1,Life Sciences,Female,1,Research Scientist,Single,-0.136563,-0.666667,1.500000,1,-0.444444,3,0.000000,0.000000,0.2,3.0,2.0,4.0,2,4
2,-0.307692,0,Travel_Frequently,Research & Development,0.833333,4,Other,Male,4,Sales Executive,Married,2.631079,-0.333333,0.166667,3,-0.555556,2,0.000000,-0.333333,0.0,2.0,2.0,1.0,3,3
3,0.153846,0,Non-Travel,Research & Development,-0.416667,5,Life Sciences,Male,3,Human Resources,Married,0.618830,0.333333,-0.500000,3,0.333333,5,0.500000,2.000000,0.4,4.0,4.0,3.0,2,3
4,-0.307692,0,Travel_Rarely,Research & Development,0.250000,1,Medical,Male,1,Sales Executive,Single,-0.474223,0.666667,-0.333333,2,-0.111111,2,0.166667,-0.333333,0.2,4.0,1.0,3.0,3,3


In [30]:
df_resultado.head()

,Age_robust,Attrition,DistanceFromHome_robust,Education,Gender,JobLevel,MonthlyIncome_robust,NumCompaniesWorked_robust,PercentSalaryHike_robust,StockOptionLevel,TotalWorkingYears_robust,YearsAtCompany_robust,YearsSinceLastPromotion_robust,YearsWithCurrManager_robust,PerformanceRating,Education_1,Education_2,Education_3,Education_4,Education_5,Gender_Female,Gender_Male,JobLevel_1,JobLevel_2,JobLevel_3,JobLevel_4,JobLevel_5,StockOptionLevel_0,StockOptionLevel_1,StockOptionLevel_2,StockOptionLevel_3,PerformanceRating_3,PerformanceRating_4,BusinessTravel,Department,EducationField,JobRole,MaritalStatus,TrainingTimesLastYear,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,1.153846,0,-0.083333,2,Female,1,1.495430,-0.333333,-0.500000,0,-1.000000,-0.666667,-0.333333,-0.6,3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.150065,0.145554,0.172814,0.144702,0.122970,0.063562,0.135181,0.110738,0.171729,0.155117
1,-0.384615,1,0.250000,1,Female,1,-0.136563,-0.666667,1.500000,1,-0.444444,0.000000,0.000000,0.2,4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.246830,0.159547,0.172814,0.177464,0.256722,0.174657,0.135181,0.169128,0.180999,0.158189
2,-0.307692,0,0.833333,4,Male,4,2.631079,-0.333333,0.166667,3,-0.555556,0.000000,-0.333333,0.0,3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.252026,0.158127,0.131440,0.167517,0.125718,0.173670,0.152379,0.170609,0.309292,0.152376
3,0.153846,0,-0.416667,5,Male,3,0.618830,0.333333,-0.500000,3,0.333333,0.500000,2.000000,0.4,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.084430,0.150178,0.160391,0.114108,0.129750,0.135027,0.130459,0.114266,0.144969,0.164485
4,-0.307692,0,0.250000,1,Male,1,-0.474223,0.666667,-0.333333,2,-0.111111,0.166667,-0.333333,0.2,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.146406,0.150178,0.167713,0.181312,0.247658,0.172906,0.130459,0.217612,0.144969,0.150151


In [34]:
df_resultado.drop(columns=['Education', 'Gender', 'JobLevel', 'StockOptionLevel', 'PerformanceRating'], inplace=True)

In [35]:
df_resultado.head()

,Age_robust,Attrition,DistanceFromHome_robust,MonthlyIncome_robust,NumCompaniesWorked_robust,PercentSalaryHike_robust,TotalWorkingYears_robust,YearsAtCompany_robust,YearsSinceLastPromotion_robust,YearsWithCurrManager_robust,Education_1,Education_2,Education_3,Education_4,Education_5,Gender_Female,Gender_Male,JobLevel_1,JobLevel_2,JobLevel_3,JobLevel_4,JobLevel_5,StockOptionLevel_0,StockOptionLevel_1,StockOptionLevel_2,StockOptionLevel_3,PerformanceRating_3,PerformanceRating_4,BusinessTravel,Department,EducationField,JobRole,MaritalStatus,TrainingTimesLastYear,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,1.153846,0,-0.083333,1.495430,-0.333333,-0.500000,-1.000000,-0.666667,-0.333333,-0.6,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.150606,0.149349,0.169076,0.142450,0.126116,0.057223,0.139531,0.112259,0.186013,0.155515
1,-0.384615,1,0.250000,-0.136563,-0.666667,1.500000,-0.444444,0.000000,0.000000,0.2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.253988,0.155722,0.170456,0.187458,0.250556,0.175960,0.143672,0.160661,0.171167,0.163491
2,-0.307692,0,0.833333,2.631079,-0.333333,0.166667,-0.555556,0.000000,-0.333333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.253988,0.155722,0.110225,0.172084,0.124242,0.172379,0.149717,0.160661,0.319268,0.152888
3,0.153846,0,-0.416667,0.618830,0.333333,-0.500000,0.333333,0.500000,2.000000,0.4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.077491,0.156102,0.160580,0.134043,0.124242,0.147694,0.127563,0.116974,0.145384,0.163018
4,-0.307692,0,0.250000,-0.474223,0.666667,-0.333333,-0.111111,0.166667,-0.333333,0.2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.149360,0.155841,0.165907,0.177812,0.258805,0.170352,0.138802,0.224400,0.146153,0.150490


In [36]:
df_resultado.to_pickle('../datos/df_encoded.pkl')